In [ ]:
import numpy as np
from random import choice as rchoice
from random import randint
import random
import cv2, traceback, imageio
import os.path as osp

from typing import Optional, List, Union, Tuple, Dict
# from utils.io_utils import imread_nogrey_rgb, json2dict
from transforms import rotate_image
# from utils.logger import LOGGER


class NameSampler:

    def __init__(self, name_prob_dict, sample_num=2048) -> None:
        self.name_prob_dict = name_prob_dict
        self._id2name = list(name_prob_dict.keys())
        self.sample_ids = []

        total_prob = 0.
        for ii, (_, prob) in enumerate(name_prob_dict.items()):
            tgt_num = int(prob * sample_num)
            total_prob += prob
            if tgt_num > 0:
                self.sample_ids += [ii] * tgt_num

        nsamples = len(self.sample_ids)
        assert prob <= 1
        if prob < 1 and nsamples < sample_num:
            self.sample_ids += [len(self._id2name)] * (sample_num - nsamples)
            self._id2name.append('_')

    def sample(self) -> str:
        return self._id2name[rchoice(self.sample_ids)]

class PossionSampler:
    def __init__(self, lam=3, min_val=1, max_val=8) -> None:
        self._distr = np.random.poisson(lam, 1024)
        invalid = np.where(np.logical_or(self._distr<min_val, self._distr > max_val))
        self._distr[invalid] = np.random.randint(min_val, max_val, len(invalid[0]))

    def sample(self) -> int:
        return rchoice(self._distr)



regular_paste_prob=0.4
person_paste_prob=0.4

paste_method_sampler = NameSampler({'regular': regular_paste_prob, 'personbbox': person_paste_prob, 
                            'partition': 1-regular_paste_prob-person_paste_prob})
    
paste_method = paste_method_sampler.sample()
print(paste_method)

numfg_sampler = PossionSampler(min_val=1, max_val=9, lam=2.5)
numfg_regpaste_sampler = PossionSampler(min_val=2, max_val=9, lam=3.5)


if paste_method == 'regular':
    num_fg = numfg_regpaste_sampler.sample()
    print(num_fg)

elif paste_method == 'personbbox':
